In [ ]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [ ]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(10)

In [ ]:
binario = df['Blemishes (Y/N)']
binario.head(30)

In [ ]:
# Caminho do dicionário dos dados .csv
csv_path = '../data/external/dicionario_de_dados_laranjas.csv'

# Leitura do CSV
df_dicionario_dados = pd.read_csv(csv_path, sep=',')
#dicionario = df_dicionario_dados.loc[:, ~df_dicionario_dados.columns.str.contains('^Unnamed')]
print(df_dicionario_dados)

In [ ]:
print(df_dicionario_dados.columns)

In [ ]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'Quality'

variavel_Binaria = (
    df_dicionario_dados
    .query("Subtipo == 'Binária' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variáveis qualitativo: Binária")
print(variavel_Binaria)
print(" ")

variavel_nominal = (
    df_dicionario_dados
    .query("Subtipo == 'Nominal' and Variável != @variavel_alvo")
    .Variável
    .to_list()
)
print(f"Variáveis qualitativo: nominal")
print(variavel_nominal)
print(" ")

variavel_quanti_continua = (
    df_dicionario_dados
    .query("Subtipo == 'Contínua' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")
variavel_quanti_discreta = (
    df_dicionario_dados
    .query("Subtipo == 'Discreto' and Variável != @variavel_alvo")
    .Variável #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Discreto")
print(variavel_quanti_discreta)
print(" ")

#X = df.drop(columns=[variavel_alvo])
#Y = df[variavel_alvo]

In [ ]:
processamento_nominal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)), # codificação de variáveis
])

processamento_ordinal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), #Tratamento de dados faltantes com substituição pela MODA
    ("encoding", OrdinalEncoder()), #Codificação de variáveis
     
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreta = Pipeline(steps=[
    ("missing", KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_binario = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False))
])


In [ ]:
pre_processador = ColumnTransformer([
    ("ordinal", processamento_ordinal, variavel_qualitativo),
    ("continuo", processamento_continuos, variavel_quanti_continua),
    ("discreta", processamento_discreta, variavel_quanti_discreta),
    ("binario", processamento_binario, variavel_Binaria
])